In [ ]:
## Task 2. Install packages and import libraries

In [1]:
# Install Vertex AI SDK & Other dependencies

!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install langchain langchain-community langchain-core langchain-google-vertexai pypdf


The following additional packages will be installed:
  liblept5 libopenjp2-7 libtesseract4 libwebpmux3 tesseract-ocr-eng
  tesseract-ocr-osd
The following NEW packages will be installed:
  liblept5 libopenjp2-7 libtesseract4 libwebpmux3 tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 7 newly installed, 0 to remove and 2 not upgraded.
Need to get 7483 kB of archives.
After this operation, 23.2 MB of additional disk space will be used.

78Selecting previously unselected package libopenjp2-7:amd64.
(Reading database ... 141592 files and directories currently installed.)
Preparing to unpack .../0-libopenjp2-7_2.4.0-3_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [  3%] [##........................................................] 8Unpacking libopenjp2-7:amd64 (2.4.0-3) ...
7Progress: [  7%] [####......................................................] 8Selecting previously unselected package libwebpmux3:amd64.

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()

app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Import libraries

import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_google_vertexai import ChatVertexAI

warnings.filterwarnings("ignore")

In [2]:
## Task 3. Load the model and prepare data

In [3]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_google_vertexai import ChatVertexAI, VertexAI, VertexAIEmbeddings

warnings.filterwarnings("ignore")

In [4]:
import os

PROJECT_ID = "[qwiklabs-gcp-04-72beeead10e0]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[qwiklabs-gcp-04-72beeead10e0]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

import vertexai

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
!gcloud ai models list --region=us-central1

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Listed 0 items.


In [6]:
#TODO: Load the pre-trained text generation model named "gemini-1.5-pro" using "ChatVertexAI" class.


# Load the Gemini 1.5 Pro model
# LLM model
llm = ChatVertexAI(
    model_name="gemini-2.0-flash",
    verbose=True,
)

my_text = "Summarize the following sentence: The quick brown fox jumps over the lazy dog.?"

llm.invoke(my_text)

AIMessage(content='The sentence describes a fox jumping over a dog.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 16, 'candidates_token_count': 11, 'total_token_count': 27, 'prompt_tokens_details': [{'modality': 1, 'token_count': 16}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.23195778239857068, 'model_name': 'gemini-2.0-flash'}, id='run--b60b0646-69a7-4f0a-9bcb-62feb08de6b6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 11, 'total_tokens': 27})

In [ ]:
#TODO: Download a PDF file from specified URL and save it in "data" directory.


In [7]:
#TODO: Download a PDF file from specified URL and save it in "data" directory.

import os
import requests

# Create the "data" directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# URL of the PDF
url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"

# Path to save the file
save_path = "data/practitioners_guide_to_mlops_whitepaper.pdf"

# Download the PDF
response = requests.get(url)
with open(save_path, "wb") as f:
    f.write(response.content)

print(f"PDF downloaded and saved to: {save_path}")

PDF downloaded and saved to: data/practitioners_guide_to_mlops_whitepaper.pdf


In [8]:
# Install Vertex AI SDK, LangChain and dependencies
%pip install --upgrade --quiet google-cloud-aiplatform langchain langchain-core langchain-text-splitters langchain-google-vertexai langchain-community faiss-cpu langchain-chroma pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 32.0.1 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain.chains import (
    ConversationChain,
    LLMChain,
    RetrievalQA,
    SimpleSequentialChain,
)
from langchain.chains.summarize import load_summarize_chain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_google_vertexai import ChatVertexAI, VertexAI, VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
!pip install -q langchain pymupdf

In [ ]:
## Task 4. Generate summaries

In [11]:
#TODO: Load the PDF file and split it into individual pages.
from langchain.document_loaders import PyMuPDFLoader

# Load the PDF file
loader = PyMuPDFLoader("data/practitioners_guide_to_mlops_whitepaper.pdf")
documents = loader.load()

print(f"Loaded {len(documents)} pages.")
print("Sample page content:\n")
print(documents[0].page_content[:500])  # Preview first page

Loaded 37 pages.
Sample page content:

Practitioners guide to MLOps:
A framework for continuous 
delivery and automation of
machine learning.
White paper
May 2021
Authors:
Khalid Salama,
Jarek Kazmierczak,
Donna Schut


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

split_docs = text_splitter.split_documents(documents)

print(f"\nSplit into {len(split_docs)} chunks.")
print("\nPreview of first chunk:\n")
print(split_docs[0].page_content)



Split into 86 chunks.

Preview of first chunk:

Practitioners guide to MLOps:
A framework for continuous 
delivery and automation of
machine learning.
White paper
May 2021
Authors:
Khalid Salama,
Jarek Kazmierczak,
Donna Schut


In [13]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document
loader = PyMuPDFLoader("data/practitioners_guide_to_mlops_whitepaper.pdf")
documents = loader.load()

# Extract just the first 3 pages
first_three_pages = documents[:3]

# Split each page into chunks if needed (optional for short pages)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
)
texts = text_splitter.split_documents(first_three_pages)

In [14]:
#Prompt design with Stuffing Chain

prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [15]:
#TODO: Set up a summarization chain using the stuff method. Incorporate the model loaded earlier into the summarization chain to enhance the quality of the summary.
from langchain.chains.summarize import load_summarize_chain

stuff_chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    prompt=prompt,
    verbose=True
)

In [16]:
summary = stuff_chain.run(texts)
print("\n--- Bullet Point Summary of First 3 Pages ---\n")
print(summary)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```Practitioners guide to MLOps:
A framework for continuous 
delivery and automation of
machine learning.
White paper
May 2021
Authors:
Khalid Salama,
Jarek Kazmierczak,
Donna Schut

Table of Contents
Executive summary	
3
Overview of MLOps lifecycle and core capabilities	
4
Deep dive of MLOps processes	
15
Putting it all together	
34
Additional resources	
36
Building an ML-enabled system	
6
The MLOps lifecycle	
7
MLOps: An end-to-end workflow	
8
MLOps capabilities	
9
      Experimentation	
11
      Data processing	
11
      Model training	
11
      Model evaluation	
12
      Model serving	
12
      Online experimentation	
13
      Model monitoring	
13
      ML pipelines	
13
      Model registry	
14
 

In [ ]:
## Task 5. Add tools to the LLM for multiplication

In [51]:
#TODO: Load the PDF file and split it into individual pages.
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

In [52]:
#TODO: Add the tools to the LLM created earlier and invoke it with the following query. Print the result in the console.
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
query = "What is 3 * 12?"
messages = [HumanMessage(query)]

In [53]:
#TODO: Iterate through the tools in the response, invoke the tools and append the response to the messages object.
llm_with_tools = llm.bind_tools(tools)
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")
query = "What is 3 * 12?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '5e96a483-83a6-4bcd-992e-184aa9a90b77',
  'type': 'tool_call'}]

In [54]:
#TODO: Invoke the LLM with the solution of the tool and the original message and print the final user response.
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is 3 * 12?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 68, 'candidates_token_count': 5, 'total_token_count': 73, 'prompt_tokens_details': [{'modality': 1, 'token_count': 68}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -3.176135942339897e-06, 'model_name': 'gemini-2.0-flash'}, id='run--c1db2769-393f-48cd-a9e4-d0d9eafc237a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '66c93742-114e-4ea3-b8f6-734595724d97', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 5, 'total_tokens': 73}),
 ToolMessage(content='36', tool_call_id='66c93742-114e